In [84]:
import numpy as np
import pandas as pd
import os
import tqdm
import torch

from itertools import chain

import matplotlib.pyplot as plt
%matplotlib inline

In [85]:
import sys

sys.path.append('..')

import utils

from importlib import reload

reload(utils)

from utils import dataset, applier, domain, pipe

reload(dataset)
reload(applier)
reload(domain)
reload(pipe)

from utils.dataset import LabeledDataset

In [3]:
applier.mobilenet_v3_small

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [4]:
data_path = "/Users/sergmiller/Documents/my/IDAO-2021/data/idao_dataset/"

In [5]:
train_path = os.path.join(data_path, "train")

In [86]:
sample_dataset = dataset.build_dataset(train_path, limit=10)

100%|██████████| 10/10 [00:00<00:00, 82.01it/s]


In [87]:
sample_dataset.save('test.npz')
sample_dataset = dataset.LabeledDataset.load('test.npz')

In [7]:
sample_dataset.samples.shape

(10, 576, 576)

In [11]:
import torchvision.models as models
import torchvision.transforms as transforms
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# model = models.mobilenet_v3_small(pretrained=True)

In [13]:
torch.set_num_threads(1)

In [14]:
torch.get_num_threads()

1

In [30]:
croped_mobilenet = torch.nn.Sequential(
    transforms.CenterCrop(64),
    applier.mobilenet_v3_small
)

In [31]:
croped_mobilenet.train(False)

Sequential(
  (0): CenterCrop(size=(64, 64))
  (1): MobileNetV3(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (relu): ReLU(inplace=True)
            (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          )
          (2): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1

In [8]:
train_dataset = dataset.build_dataset(train_path)

100%|██████████| 13404/13404 [01:37<00:00, 136.97it/s]


In [88]:
train_emb_dataset = applier.build_embd_dataset(train_dataset, croped_mobilenet)

100%|██████████| 838/838 [01:14<00:00, 11.28it/s]


In [102]:
train_emb_dataset.save('../data/center_crop_64_mobilenet_v3_small_embeddings.npz')

In [154]:
train_emb_dataset = LabeledDataset.load('../data/mobilenet_v3_small_embeddings.npz')

In [90]:
train_emb_dataset.samples.shape

(13404, 1000)

In [103]:
def apply_random_projection_and_normalize(data : np.array, to_dim : int = 10, seed : int = 0):
    assert 2 == len(data.shape)
    np.random.seed(seed)
    N = data.shape[1]
    proj_matrix = np.random.randn(N, to_dim) / np.sqrt(N)
    print(np.mean(proj_matrix), np.std(proj_matrix))
    data_proj = np.matmul(data, proj_matrix)
    data_proj /= np.sum(np.abs(data_proj), axis=1, keepdims=True)
    return data_proj

In [104]:
import copy
proj_emb_dataset = copy.copy(train_emb_dataset)
proj_emb_dataset.samples = apply_random_projection_and_normalize(
    train_emb_dataset.samples, to_dim=50)

-0.00011997710679547996 0.03151695476906101


In [188]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [192]:
build_fold = lambda seed: StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
params = {
    "penalty": 'l1',
    "solver": 'saga',
    "verbose": 10,
    "max_iter": 100,
    "tol": 1e-5,
    "scoring" : 'roc_auc',
    "random_state": 0,
    "cv": build_fold(0),
    "n_jobs": 7,
    "refit": False
}

In [149]:
ids = np.arange(len(train_proj_emb_dataset.samples))
np.random.seed(0)
np.random.shuffle(ids)
N = int(len(ids) * 0.9)
train_ids = ids[:N]
val_ids = ids[N:]
print(train_ids[:10], val_ids[:10])

[10869  3245 13299 10980  1103  5091 10134  6859  4516  4422] [ 1057 10013  6979  7505    60  6173  4878    35 11686  7315]


In [164]:
train_proj_emb_dataset = proj_emb_dataset.subset(train_ids)
val_proj_emb_dataset = proj_emb_dataset.subset(val_ids)

In [193]:
cv_label1 = LogisticRegressionCV(**params).fit(
    X=proj_emb_dataset.samples,
    y=(proj_emb_dataset.labels[:, 0] == 'NR').astype(int))

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds

convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds



convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds

convergence after 1 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 secondsconvergence after 3 epochs took 0 seconds

convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
convergence after 19 epochs took 1 seconds
convergence after 20 epochs took 1 seconds
convergence after 23 epochs took

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 1 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not 

...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 0 seconds
max_iter reached after 0 seconds
max_iter reached after 1 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 1 seconds
max_iter reached after 1 seconds
max_iter reached after 0 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:   15.1s finished


In [194]:
np.mean(cv_label1.scores_[1],axis=0)  # with crop64 normalized random cv=10

array([0.5       , 0.5       , 0.5       , 0.70546698, 0.9175677 ,
       0.95809902, 0.96694732, 0.97029197, 0.9720148 , 0.97301826])

In [111]:
np.mean(cv_label1.scores_[1],axis=0)  # with crop64 normalized

array([0.5       , 0.5       , 0.5       , 0.70550071, 0.91753139,
       0.95811081, 0.96693674, 0.97036226, 0.97211698, 0.97315119])

In [96]:
np.mean(cv_label1.scores_[1],axis=0)  # with crop64

array([0.5       , 0.78958939, 0.93599517, 0.96792367, 0.97494673,
       0.97596747, 0.97633959, 0.97650932, 0.97657727, 0.97661092])

In [127]:
np.mean(cv_label1.scores_[1], axis=0)  # new

array([0.5       , 0.5       , 0.76982608, 0.86866077, 0.9326519 ,
       0.94429291, 0.94827148, 0.95034892, 0.95169425, 0.9526548 ])

In [124]:
np.mean(cv_label1.scores_[1], axis=0)  # old

array([0.5       , 0.5       , 0.7952889 , 0.87402804, 0.94469283,
       0.95739159, 0.96157609, 0.96381906, 0.96529288, 0.96639186])

In [112]:
cv_label1.get_params()

{'Cs': 10,
 'class_weight': None,
 'cv': 10,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1.0,
 'l1_ratios': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': 7,
 'penalty': 'l1',
 'random_state': 0,
 'refit': True,
 'scoring': 'roc_auc',
 'solver': 'saga',
 'tol': 1e-05,
 'verbose': 10}

In [170]:
from sklearn.linear_model import LassoCV, LassoLarsCV, LogisticRegressionCV

In [162]:
!pip install catboost
import catboost

     |████████████████████████████████| 11.1 MB 8.8 MB/s eta 0:00:01    |███                             | 1.1 MB 2.3 MB/s eta 0:00:05
  Using cached graphviz-0.16-py2.py3-none-any.whl (19 kB)
     |████████████████████████████████| 13.2 MB 3.6 MB/s eta 0:00:01
  Using cached retrying-1.3.3.tar.gz (10 kB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=7142ea69f2bdc9e51212c74b906d77f6f989793d634af1c973839d25de34a510
  Stored in directory: /Users/sergmiller/Library/Caches/pip/wheels/ce/18/7f/e9527e3e66db1456194ac7f61eb3211068c409edceecff2d31
Successfully built retrying
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [195]:
import copy

classes = [1, 10, 20, 3, 30, 6]

def mae_scorer(estimator, X, y):
#     y_pred = np.array([sum(z * classes[i] for i,z in enumerate(_y)) for _y in estimator.predict_proba(X)])
    y_pred = np.array([classes[int(_y)] for _y in estimator.predict(X)])
    y = np.array([classes[_y] for _y in y])
    return -np.mean(np.abs(y_pred - y))

params2 = copy.copy(params)

params2['cv'] = build_fold(1)
params2['scoring'] = mae_scorer
# params2['class_weight'] = {str(c) : c for c in classes}
cv_label2 = LogisticRegressionCV(**params2).fit(
    proj_emb_dataset.samples,
    proj_emb_dataset.labels[:, 1])

[Parallel(n_jobs=7)]: Using backend ThreadingBackend with 7 concurrent workers.


convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds


convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds


convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 secondsconvergence after 1 epochs took 0 seconds

convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 1 epochs took 0 seconds
convergence after 3 epochs took 0 seconds
convergence after 3 epochs took 0 secondsconvergence after 3 epochs took 0 seconds

convergence after 3 epochs took 0 seconds
convergence after 4 epochs took 0 seconds
convergence after 56 epochs took 3 seconds
convergence after 56 epochs took 3 seconds
convergence after 61 epochs took

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 4 secondsmax_iter reached after 4 seconds

max_iter reached after 4 seconds
max_iter reached after 4 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


...

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 3 seconds
max_iter reached after 3 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 3 seconds


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:   53.4s finished


In [196]:
np.mean(cv_label2.scores_['1'],axis=0) # crop64 proj normalized random cv=10

array([-10.92192026, -12.54789041, -10.79639387,  -3.9197257 ,
        -1.48255456,  -0.80103977,  -0.60990228,  -0.56424839,
        -0.53224003,  -0.51590042])

In [172]:
np.mean(cv_label2.scores_['1'],axis=0) # crop64 proj normalized

array([-17.42117233, -10.81699934,  -8.40923982,  -3.90688966,
        -1.49081772,  -0.79319794,  -0.61772669,  -0.56096464,
        -0.53365644,  -0.51455953])

In [136]:
np.mean(cv_label2.scores_['1'],axis=0) # crop64 proj normalized prob weighted

array([-10.76203842,  -9.08192111,  -8.67627585,  -7.93668639,
        -3.32040689,  -1.61058029,  -1.13415802,  -0.98432688,
        -0.91080838,  -0.86552122])

In [114]:
np.mean(cv_label2.scores_['1'],axis=0) # crop64 proj normalized

array([-17.42117233, -10.81699934,  -8.40923982,  -3.90688966,
        -1.49081772,  -0.79319794,  -0.61772669,  -0.56096464,
        -0.53365644,  -0.51455953])

In [46]:
np.mean(cv_label2.scores_['1'],axis=0) # crop64 proj

array([-8.3034167 , -2.87024781, -1.16681787, -0.68667234, -0.50216841,
       -0.46188559, -0.43882784, -0.42890286, -0.42718655, -0.43352677])

In [24]:
np.mean(cv_label2.scores_['1'],axis=0) # mini weighted

array([7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.7, 5.7, 5.7, 5.7])

In [130]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted proj

array([-9.31475597, -8.29894242, -2.84744388, -1.16951128, -0.74142058,
       -0.62891488, -0.60055984, -0.58071633, -0.56407921, -0.55960338])

In [142]:
np.mean(cv_label2.scores_['1'],axis=0) # not weighted

array([10.68017027, 10.68017027, 10.09729052,  9.80483741,  9.63690119,
        9.57527796,  9.54491318,  9.53208126,  9.52514296,  9.51149031])

In [144]:
np.mean(cv_label2.scores_['1'],axis=0) # weighted

array([ 9.59965186,  9.4570564 ,  9.96051226, 10.08321655,  9.88160343,
        9.84861552,  9.80512292,  9.85751204,  9.71321238,  9.77365697])

In [47]:
pub_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "public_test"),
    sample_processor=domain.process_test_sample)

100%|██████████| 1502/1502 [00:10<00:00, 140.18it/s]


In [48]:
private_test_dataset = dataset.build_dataset(
    os.path.join(data_path, "private_test"),
    sample_processor=domain.process_test_sample)

100%|██████████| 15062/15062 [01:50<00:00, 135.94it/s]


In [197]:
class ModelWithProjection:
    def __init__(self, model):
        self.model = model
    def predict(self, x):
        x_proj = apply_random_projection_and_normalize(x, to_dim=50)
        return self.model.predict(x_proj)
    def predict_proba(self, x):
        x_proj = apply_random_projection_and_normalize(x, to_dim=50)
        return self.model.predict_proba(x_proj)

In [198]:
model1 = ModelWithProjection(cv_label1)
model2 = ModelWithProjection(cv_label2)

In [199]:
sample_res = pipe.apply_all_models_to_test_dataset(
    sample_dataset,
    croped_mobilenet,
    model1,
    model2,
    'sample')

-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [176]:
sample_res.tags.shape, sample_res.samples.shape

((10,), (10, 1000))

In [207]:
pub_test_predictions = pipe.apply_all_models_to_test_dataset(
    pub_test_dataset, croped_mobilenet, model1, model2, 'pub_test')

100%|██████████| 94/94 [00:08<00:00, 10.53it/s]


-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [201]:
private_test_predictions = pipe.apply_all_models_to_test_dataset(
    private_test_dataset, croped_mobilenet, model1, model2, 'private_test')

-0.00011997710679547996 0.03151695476906101
-0.00011997710679547996 0.03151695476906101


In [101]:
pub_test_predictions.save('../data/pub_test_predictions_embeds_crop_64_proj_50.npz')
private_test_predictions.save('../data/private_test_predictions_embeds_crop_64_proj_50.npz')

In [202]:
pub_test_predictions.labels[:20]

array([['0.5604101123351646', '3'],
       ['0.9985373566322635', '30'],
       ['0.9668058357753039', '30'],
       ['0.9985136413307789', '30'],
       ['0.1710724280566548', '6'],
       ['0.048769952472209166', '20'],
       ['0.8355921261079253', '30'],
       ['0.00579688676606116', '20'],
       ['0.9313381187356394', '3'],
       ['0.02248359732964944', '1'],
       ['0.5037619511742377', '6'],
       ['0.04114410330929896', '1'],
       ['0.9999972643512921', '30'],
       ['0.9781792038400923', '3'],
       ['0.9999744067424009', '30'],
       ['0.009448060058281027', '20'],
       ['0.0071320900544105115', '20'],
       ['0.7462290395413042', '3'],
       ['0.5126432089800697', '3'],
       ['0.9288328758610631', '3']], dtype='<U32')

In [203]:
private_test_predictions.labels[:20]

array([['0.9959382927115767', '10'],
       ['0.34786341947131505', '20'],
       ['0.05196240581725098', '1'],
       ['0.00018201494640013305', '20'],
       ['0.05203463857686008', '1'],
       ['0.2177242842992685', '6'],
       ['0.9999999998696617', '30'],
       ['0.9718452052039402', '3'],
       ['0.9211651566940707', '10'],
       ['0.7965147791960825', '3'],
       ['0.058037786211757036', '1'],
       ['0.5909350742715967', '3'],
       ['6.420147295393974e-05', '20'],
       ['0.30202234373113057', '6'],
       ['0.034833417597421645', '20'],
       ['0.0052965988099766825', '20'],
       ['0.9999980441823341', '30'],
       ['0.00032812345435850876', '20'],
       ['0.00017769293317138057', '20'],
       ['0.02835726140848327', '1']], dtype='<U32')

In [70]:
train_emb_dataset.labels[7]

array(['NR', '20'], dtype='<U21')

In [71]:
model1.predict_proba(train_emb_dataset.samples[4].reshape(1, -1))

-0.00011997710679547996 0.03151695476906101


array([[0.03249416, 0.96750584]])

In [64]:
model2.predict_proba(train_emb_dataset.samples[2].reshape(1, -1))

-0.00011997710679547996 0.03151695476906101


array([[1.65388373e-22, 1.10186400e-02, 9.88966091e-01, 1.22186016e-11,
        1.52692594e-05, 6.21623352e-11]])

In [204]:
submit = dataset.LabeledDataset.merge(
    pub_test_predictions, private_test_predictions)

In [198]:
submit.labels[:, 0] = np.array([1 - float(x) for x in submit.labels[:, 0]])

In [205]:
submit.labels[:10]

array([['0.5604101123351646', '3'],
       ['0.9985373566322635', '30'],
       ['0.9668058357753039', '30'],
       ['0.9985136413307789', '30'],
       ['0.1710724280566548', '6'],
       ['0.048769952472209166', '20'],
       ['0.8355921261079253', '30'],
       ['0.00579688676606116', '20'],
       ['0.9313381187356394', '3'],
       ['0.02248359732964944', '1']], dtype='<U32')

In [206]:
dataset.dataset2submit_csv(submit,
    "../submits/1:log_reg_2:log_reg_multiclass_not_refit_over_proj50_normalized_crop_64_seed_0_mobilenet_v3_small_embds_shuffled_kfold_10.csv")

In [127]:
submit.tags.shape

(16564,)

In [128]:
submit.samples.shape

(16564, 1000)

In [129]:
pub_test_predictions.tags.shape, pub_test_predictions.samples.shape

((1502,), (1502, 1000))

In [130]:
private_test_predictions.tags.shape, private_test_predictions.samples.shape

((15062,), (15062, 1000))

In [244]:
model = applier.mobilenet_v3_small
model.train(False)

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [245]:
torch.save(model.state_dict(), "mobilenet_state_dict")

In [246]:
import torchvision
model2 = mobilenet_v3_small()

In [247]:
model2.load_state_dict(torch.load("mobilenet_state_dict"))

<All keys matched successfully>

In [248]:
np.random.seed(0)
t = np.random.random((1,3,500,500))
model2.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = model2(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))

0.5002803215266387 0.2887019402063319
0.46069193 1.7000295


In [249]:
np.random.seed(0)
t = np.random.random((1,3,500,500))
model.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = model(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))

0.5002803215266387 0.2887019402063319
0.46069193 1.7000295


In [214]:
model

MobileNetV3(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU(inplace=True)
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (2): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_r

In [231]:
import torchvision
mobilenet = torchvision.models.mobilenet_v3_small()
mobilenet.load_state_dict(torch.load("mobilenet_state_dict"))

np.random.seed(0)
t = np.random.random((1,3,500,500))
model2.train(False)
print(np.mean(t), np.std(t))
with torch.no_grad():
    res = mobilenet(torch.Tensor(t)).detach().numpy()
print(np.mean(res), np.std(res))



import time
start = time.time()
for i in np.arange(10):
    t = np.random.random((32,3,500,500))
    with torch.no_grad():
        res = mobilenet(torch.Tensor(t)).detach().numpy()
    
print('OK, run {}s'.format(time.time() - start))

0.5002803215266387 0.2887019402063319
3.4595482 0.81053764
OK, run 25.1478328704834s


In [242]:
import sys

sys.path.append('..')
from submition_track2.test_in_submit.mobilenet_v3 import mobilenet_v3_small

In [243]:
mobilenet = mobilenet_v3_small()